In [ ]:
import pandas as pd
data=pd.read_csv("data.csv")
data.head()

,Question,Answer
0,पौधे क्या उत्सर्जित करते हैं,रात में कार्बन
1,ISRO में S का अर्थ क्या है,Space
2,इड्डियों एवं दाँतों में मुख्य रूप से कौन-सा रस...,कैल्शियम
3,रसोई गैस का मिश्रण क्या है,ब्यूटेन एवं प्रोपेन का
4,‘हीमोफीलिया’ एक आनुवंशिक रोग है; जिसका क्या पर...,रक्त का नहीं जमना


In [ ]:
def getResults(questions, fn):# function for returning the output answer if found along with similarity score
    def getResult(q):
        answer, score, prediction = fn(q)
        return [q, prediction, answer, score]
    return pd.DataFrame(list(map(getResult, questions)), columns=["Q", "Prediction", "A", "Score"])
test_data = [
   "हिरा क्यों चमकता है",
   "ISRO का फुल फॉर्म क्या है",
   "रक्त लाल रंग का क्यों होता है ",
   "भारत की राजधानी क्या है ",
   "मानव शरीर में कितनी हड्डियां पायी जाती हैं ",
   "२६ जनवरी कौन सा दिवस है ",
   "पिंक सिटी",
   "भारत के प्रधानमंत्री कौन हैं ",
   "पंजाब में कुल कितनी नदियां हैं ",
   "सूर्य के बाद आने वाला गृह "
]

In [ ]:
import numpy as np
def ratio(s, t, ratio_calc = False):#Calculates levenshtein distance between two strings.
    
    # defined matrix of zeros to hold similarity scores
    rows = len(s)+1
    cols = len(t)+1
    distance = np.zeros((rows,cols),dtype = int)

    # initialized the matrix with the indeces of each character of both strings
    for i in range(1, rows):
        for k in range(1,cols):
            distance[i][0] = i
            distance[0][k] = k

    # Iterating over the matrix to compute the cost of deletions,insertions and substitutions    
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0 # If the characters are the same in the two strings 
            else:
                if ratio_calc == True:
                    cost = 2 #if ratio is calculated cost of substitution is 2 according to python levenshtein package
                else:
                    cost = 1 # for distance cost of substitution is 1.
            distance[row][col] = min(distance[row-1][col] + 1,      # Cost of deletions
                                 distance[row][col-1] + 1,          # Cost of insertions
                                 distance[row-1][col-1] + cost)     # Cost of substitutions
    if ratio_calc == True:
        # Computation of the Levenshtein Distance Ratio
        Ratio = ((len(s)+len(t)) - distance[row][col]) / (len(s)+len(t))
        return Ratio
    else:
        # minimum number of edits required to convert string s to string t
        return "The strings are {} edits away".format(distance[row][col])

In [ ]:
def getApproximateAnswer(q):
    max_score = 0
    answer = ""
    prediction = ""
    for idx, row in data.iterrows():
        score = ratio(row["Question"], q,ratio_calc = True)
        if score > 0.9: # maximum score if found return the answer
            return row["Answer"], score, row["Question"]
        elif score > max_score: # new maximum score found
            max_score = score
            answer = row["Answer"]
            prediction = row["Question"]
    if max_score > 0.6: # checking if the maxscore is greter than threshold score
        return answer, max_score, prediction
    return "Sorry, I didn't get you.", max_score, prediction
getResults(test_data, getApproximateAnswer)

,Q,Prediction,A,Score
0,हिरा क्यों चमकता है,हीरा किस कारण चमकता है,पूर्ण आतंरिक परावर्तन के कारण,0.682927
1,ISRO का फुल फॉर्म क्या है,MCB का फुल फॉर्म क्या है,Miniature Circuit Breaker,0.857143
2,रक्त लाल रंग का क्यों होता है,रक्त में लाल रंग किसके कारण होता है,7 हीमोग्लोबिन,0.738462
3,भारत की राजधानी क्या है,गुजरात की राजधानी है,गांधीनगर,0.727273
4,मानव शरीर में कितनी हड्डियां पायी जाती हैं,मानव शरीर में कितनी हड्डी होती है?,206,0.805195
5,२६ जनवरी कौन सा दिवस है,जंग लगना कौन सा परिवर्तन है,"Sorry, I didn't get you.",0.588235
6,पिंक सिटी,पिंक सिटी,जयपुर,1.000000
7,भारत के प्रधानमंत्री कौन हैं,भारत के प्रथम सिख प्रधानमंत्री कौन है,मनमोहन सिंह,0.818182
8,पंजाब में कुल कितनी नदियां हैं,भारत में कुल कितने उच्च न्यायालय हैं,21,0.656716
9,सूर्य के बाद आने वाला गृह,सूर्य से सबसे नजदीक गृह है-,बुध,0.603774
